In [2]:
!pip install -q --upgrade pip
!pip install -U jaxlib==0.4.4+cuda11.cudnn82 -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
!pip install -q --upgrade jax[cuda11_cudnn805] -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
!pip install -q git+https://www.github.com/google/neural-tangents
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cpu
!pip install pyg-lib torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.13.0+cpu.html

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.8/112.8 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.7/79.7 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 55.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.24.2 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.de

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
%cd /content/drive/MyDrive/Colab Notebooks/Masterarbeit/two_WL_NTK

/content/drive/MyDrive/Colab Notebooks/Masterarbeit/two_WL_NTK


In [9]:
from utils import *
from data_preperation import data_preperation, twl_sparse_pattern, twl_sparse_edge_features
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
import os
import shutil
from jax import numpy as jnp
import numpy as np
import jax

In [4]:
d = TUDataset(root="Masterarbeit", name="MUTAG")

In [3]:
nb_nodes = 3

edges = jnp.array([[0,0],
                   [1,1],
                   [2,2],
                   [0,1],
                   [0,2],
                   [1,2]])

node_features = jnp.array([[1,0,0],
                         [0,1,0],
                         [0,0,1],])

pattern, edge_list = twl_sparse_pattern(edges, nb_nodes)
edge_features = twl_sparse_edge_features(node_features, edge_list.shape[0], nb_nodes)

In [4]:
pattern

Array([[0, 0, 0],
       [0, 3, 3],
       [0, 4, 4],
       [1, 1, 1],
       [1, 3, 3],
       [1, 5, 5],
       [2, 2, 2],
       [2, 4, 4],
       [2, 5, 5],
       [3, 0, 3],
       [3, 3, 1],
       [3, 4, 5],
       [4, 0, 4],
       [4, 3, 5],
       [4, 4, 2],
       [5, 1, 5],
       [5, 3, 4],
       [5, 5, 2]], dtype=int32)

In [5]:
edge_features

Array([[1., 0., 0., 1.],
       [0., 1., 0., 1.],
       [0., 0., 1., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.]], dtype=float32)

In [6]:
edge_list.shape

(6, 2)

In [8]:
#  kernel pattern
# the edge list must be reshaped to (1, #edges in edge list, 1, feature dim)
# the kernel dimension after the first convolution is (1, 1, 1, 1, #edges in edge list, #edges in edge list) 

nb_edges = edge_list.shape[0]

patterns = row_wise_karthesian_prod(pattern, pattern)
# i_j, i_a, a_j, ib_jb, ib_ab, ab_jb
#   0,   1,   2,     3,     4,    5
 
e_i_j_ib_jb = patterns[:,[0,3]]
e_i_a_ib_ab = patterns[:,[1,4]] 
e_i_a_ab_jb = patterns[:,[1,5]]
e_a_j_ib_ab = patterns[:,[2,4]]
e_a_j_ab_jb = patterns[:,[2,5]]

e_i_j_ib_jb = jnp.ravel_multi_index([e_i_j_ib_jb[:,0], e_i_j_ib_jb[:,1]], (nb_edges, nb_edges)) 
e_i_a_ib_ab = jnp.ravel_multi_index([e_i_a_ib_ab[:,0], e_i_a_ib_ab[:,1]], (nb_edges, nb_edges))  
e_i_a_ab_jb = jnp.ravel_multi_index([e_i_a_ab_jb[:,0], e_i_a_ab_jb[:,1]], (nb_edges, nb_edges)) 
e_a_j_ib_ab = jnp.ravel_multi_index([e_a_j_ib_ab[:,0], e_a_j_ib_ab[:,1]], (nb_edges, nb_edges)) 
e_a_j_ab_jb = jnp.ravel_multi_index([e_a_j_ab_jb[:,0], e_a_j_ab_jb[:,1]], (nb_edges, nb_edges)) 


theta_i_a_ib_ab = jax.ops.segment_sum(np.take(k.ntk, e_i_a_ib_ab), e_i_j_ib_jb, num_segments)
theta_i_a_ab_jb = jax.ops.segment_sum(np.take(k.ntk, e_i_a_ab_jb), e_i_j_ib_jb, num_segments)
theta_a_j_ib_ab = jax.ops.segment_sum(np.take(k.ntk, e_a_j_ib_ab), e_i_j_ib_jb, num_segments)
theta_a_j_ab_jb = jax.ops.segment_sum(np.take(k.ntk, e_a_j_ab_jb), e_i_j_ib_jb, num_segments)

Array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35], dtype=int32)

In [55]:
linear_indx = ravel_multi_index_upper(edges_full_prod[:,0], edges_full_prod[:,1], nb_nodes, nb_nodes)
linear_indx


Array([0, 4, 1, 8, 2, 5, 1, 4, 2, 5, 8, 5, 2, 8, 0, 0, 1, 4], dtype=int32)

In [52]:
# edge features sorted not with self edges first

nodes_features = jnp.array(d[0].x)
print(nodes_features.shape)
print(edges.shape)

# the index for the self edges
a = jnp.array(range(nb_nodes))
linear_indx = ravel_multi_index_upper(a, a, nb_nodes, nb_nodes)

edge_features = jnp.zeros((edges.shape[0], nodes_features.shape[1]+1))

# "BIAS" feature
edge_features = edge_features.at[:,-1].set(1)

# self edges get node feature
edge_features = edge_features.at[linear_indx,:-1].set(nodes_features)

print(edge_features.shape)

edge_features

(17, 7)
(6, 2)


ValueError: ignored

In [44]:
ravel_multi_index_upper(edges_full_prod[:,[0]], edges_full_prod[:,[1]], nb_nodes, nb_nodes)

Array([[ 0],
       [18],
       [ 1],
       [36],
       [ 2],
       [19],
       [ 1],
       [18],
       [ 2],
       [19],
       [36],
       [19],
       [ 2],
       [36],
       [ 0],
       [ 0],
       [ 1],
       [18]], dtype=int32)

In [40]:
a

Array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16],      dtype=int32)

In [230]:
jnp.array(d[0].x)

Array([[1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0.]], dtype=float32)

In [ ]:
# TODO datasets:
# MUTAG -> TUDataset(root="Masterarbeit", name="MUTAG")
# PTC -> TUDataset(root="Masterarbeit", name="PTC_MR")
# NCI1 -> TUDataset(root="Masterarbeit", name="NCI1")
# PROTEINS -> TUDataset(root="Masterarbeit", name="PROTEINS")
# COLLAB -> TUDataset(root="Masterarbeit", name="COLLAB")
# IMDB-BINARY -> TUDataset(root="Masterarbeit", name="IMDB-BINARY")
# IMDB-MULTI -> TUDataset(root="Masterarbeit", name="IMDB-MULTI")

# one dataset from https://ogb.stanford.edu/docs/home/

# ogbg-molpcba https://ogb.stanford.edu/docs/graphprop/#ogbg-mol

# ogbg-ppa https://ogb.stanford.edu/docs/graphprop/#ogbg-ppa

In [ ]:
#                 gs  cl  AN     AE             spars edge features
# MUTAG        |  188	2	 17.93	19.79            32 *    188*20 * fea. dim    120320/1000000000     0.00012032  
# PTC          |  344 2	 14.29	14.69            32 *    344*15 * fea. dim    165120/1000000000     0.00016512  
# NCI1         | 4110	2	 29.87	32.30            32 *   4110*33 * fea. dim    4340160/1000000000    0.00434016 
# PROTEINS     | 1113	2	 39.06	72.82            32 *   1113*73 * fea. dim    2599968/1000000000    0.002599968  
# COLLAB       | 5000	3	 74.49	2457.78          32 * 5000*2458 * fea. dim    393280000/1000000000  0.39328  
# IMDB-BINARY  | 1000	2	 19.77	96.53            32 *   1000*97 * fea. dim    3104000/1000000000    0.003104
# IMDB-MULTI   | 1500	3	 13.00	65.94            32 *   1500*66 * fea. dim    3168000/1000000000    0.003168

In [ ]:
print(32 * 188*20)
print(32 * 344*15)
print(32 * 4110*33)
print(32 * 1113*73)
print(32 * 5000*2458)
print(32 * 1000*97)
print(32 * 1500*66)

120320
165120
4340160
2599968
393280000
3104000
3168000


In [ ]:
print(120320/1000000000)
print(165120/1000000000)
print(4340160/1000000000)
print(2599968/1000000000)
print(393280000/1000000000)
print(3104000/1000000000)
print(3168000/1000000000)

0.00012032
0.00016512
0.00434016
0.002599968
0.39328
0.003104
0.003168


In [ ]:
# Load all TU datsets

tu_datasets = ["MUTAG", "PTC", "NCI1", "PROTEINS", "COLLAB", "IMDB-BINARY", "IMDB-MULTI"]

for dataset_name in tu_datasets[2:]:

  print(f"Preparing {dataset_name} dataset:")

  dataset = TUDataset(root="Masterarbeit", name=dataset_name)

  # make lists of dense adjacency matrix, node feature array and y
  edge_index = list()
  graps_node_features = list()
  ys = list()
  for data in dataset:
      edge_index.append(data.edge_index)
      graps_node_features.append(np.array(data.x))
      ys.append(np.array(data.y))

  del dataset

  # save preprocessed data
  # creates the following files in the base path directory
  # ys.npy
  # graps_node_features.npy
  # graphs_edge_features.npy
  # graph_conv_pattern.npy
  # two_wl_pattern_radius_1.npy, two_wl_pattern_radius_2.npy, ..., two_wl_pattern_radius_k.npy
  two_wl_radius = [1,2,4]
  base_path = f"/content/drive/MyDrive/MasterarbeitData/Preprocessed/{dataset_name}"
  graps_edge_features = None
  edge_featur_init = "BIAS"

  if not os.path.exists(base_path):
      os.makedirs(base_path)

  data_preperation(edge_index, graps_node_features, graps_edge_features, edge_featur_init, ys, dataset_name, base_path, two_wl_radius)

Preparing NCI1 dataset:


NameError: ignored

In [ ]:
# Load all TU datsets

tu_datasets = ["MUTAG", "PTC", "NCI1", "PROTEINS", "COLLAB", "IMDB-BINARY", "IMDB-MULTI"]

dataset_name = "NCI1"

dataset = TUDataset(root="Masterarbeit", name=dataset_name)

# make lists of dense adjacency matrix, node feature array and y
edge_index = list()
graps_node_features = list()
ys = list()
for data in dataset:
    edge_index.append(data.edge_index)
    graps_node_features.append(np.array(data.x))
    ys.append(np.array(data.y))

del dataset

In [ ]:
# save preprocessed data
# creates the following files in the base path directory
# ys.npy
# graps_node_features.npy
# graphs_edge_features.npy
# graph_conv_pattern.npy
# two_wl_pattern_radius_1.npy, two_wl_pattern_radius_2.npy, ..., two_wl_pattern_radius_k.npy
two_wl_radius = [1,2,4]
base_path = f"/content/drive/MyDrive/MasterarbeitData/Preprocessed/{dataset_name}"
graps_edge_features = None
edge_featur_init = "BIAS"

if not os.path.exists(base_path):
    os.makedirs(base_path)

data_preperation(edge_index, graps_node_features, graps_edge_features, edge_featur_init, ys, dataset_name, base_path, two_wl_radius)

Preparing edge features
create a array of shape 4110 111 111 37
